# Unidad 04

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

## Busco obtener lo mismo que en esta tabla
|IDCliente|IDPedido|val       |pctcliente|
|---------|--------|----------|----------|
|ALFKI|10643|814.50|19.06|
|ALFKI|10692|878.00|20.55|
|ALFKI|10702|330.00|7.72|
|ALFKI|10835|845.80|19.79|
|ALFKI|10952|471.20|11.03|
|ALFKI|11011|933.50|21.85|

In [ ]:
SELECT IDCliente, IDPedido, val
FROM ValoresPedido
WHERE IDCliente = 'ALFKI'; 

SELECT SUM(val) as Total
FROM ValoresPedido
WHERE IDCliente = 'ALFKI'
;

In [ ]:
SELECT IDCliente, IDPedido, val, 
CAST(val/(SELECT SUM(val) FROM ValoresPedido WHERE IDCliente = 'ALFKI')*100.0 AS NUMERIC(5,2)) AS Porcentaje
FROM ValoresPedido
WHERE IDCliente = 'ALFKI'; 

## Funciones Aggregate de Ventana

In [ ]:
USE Pampero;
SELECT IDCliente, IDPedido, val,
SUM(val) OVER(PARTITION BY IDCliente) AS clientotal,
SUM(val) OVER() AS totalgeneral
FROM ValoresPedido;

In [ ]:
SELECT IDCliente, IDPedido, val,
CAST(100.0*val/SUM(val) OVER(PARTITION BY IDCliente) AS NUMERIC(5,2)) AS pctcliente,
CAST(100.0*val/SUM(val) OVER() AS NUMERIC(5,2)) AS pcttotal
FROM ValoresPedido;

## Enmarcado

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val,
SUM(val) OVER(PARTITION BY IDCliente
    ORDER BY FechaPedido, IDPedido
    ROWS BETWEEN UNBOUNDED PRECEDING
        AND CURRENT ROW) AS totalacumulado
FROM ValoresPedido;

In [ ]:
WITH TotalesAcumulados AS
(
SELECT IDCliente, IDPedido, FechaPedido, val,
SUM(val) OVER(PARTITION BY IDCliente
    ORDER BY FechaPedido, IDPedido
    ROWS BETWEEN UNBOUNDED PRECEDING
        AND CURRENT ROW) AS totalacumulado
FROM ValoresPedido
)
SELECT *
FROM TotalesAcumulados
WHERE totalacumulado < 1000.00;

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val,
SUM(val) OVER(PARTITION BY IDCliente
                ORDER BY FechaPedido, IDPedido
                ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS totalacumulado
FROM ValoresPedido;

## Funciones Ranking de Ventana

In [ ]:
SELECT IDCliente, IDPedido, val,
    ROW_NUMBER() OVER(ORDER BY val) AS rownum,
    RANK() OVER(ORDER BY val) AS rnk,
    DENSE_RANK() OVER(ORDER BY val) AS densernk,
    NTILE(100) OVER(ORDER BY val) AS ntile100
FROM ValoresPedido;

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val,
LAG(val,1,0) OVER(PARTITION BY IDCliente
                ORDER BY FechaPedido, IDPedido) AS prev_val,
LEAD(val,1,0) OVER(PARTITION BY IDCliente
                ORDER BY FechaPedido, IDPedido) AS next_val
FROM ValoresPedido;

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val,
FIRST_VALUE(val) OVER(PARTITION BY IDCliente
                        ORDER BY FechaPedido, IDPedido
                        ROWS BETWEEN UNBOUNDED PRECEDING
                            AND CURRENT ROW) AS primer_val,
LAST_VALUE(val) OVER(PARTITION BY IDCliente
                        ORDER BY FechaPedido, IDPedido
                        ROWS BETWEEN CURRENT ROW
                            AND UNBOUNDED FOLLOWING) AS ultimo_val
FROM ValoresPedido
ORDER BY IDCliente, FechaPedido, IDPedido;

In [ ]:
SELECT IDPedido, IDProducto, Cantidad
    ,SUM(Cantidad) OVER (PARTITION BY IDPedido ORDER BY IDPedido, IDProducto) AS Total
    ,AVG(Cantidad) OVER (PARTITION BY IDPedido ORDER BY IDPedido, IDProducto) AS Promedio
    ,COUNT(Cantidad) OVER (PARTITION BY IDPedido ORDER BY IDPedido, IDProducto) AS Cantidad
FROM [Detalles Pedido]
WHERE IDPedido IN(10847,10979);

In [ ]:
USE Pampero;
SELECT IDPedido, IDProducto, Cantidad
    ,SUM(Cantidad) OVER ven1 AS Total
    ,AVG(Cantidad) OVER ven1 AS Promedio
    ,COUNT(Cantidad) OVER ven1 AS Cantidad
FROM [Detalles Pedido]
WHERE IDPedido IN(10847,10979)
WINDOW ven1 AS (PARTITION BY IDPedido ORDER BY IDPedido, IDProducto);
